<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>☢️Radon</font>](https://www.kaggle.com/c/11mar24jh-radon/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle >> log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >>log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log       # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 11mar24jh-radon # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log              # download competition dataset as a zip file
!unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show           # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 11mar24jh-radon
Using competition: 11mar24jh-radon
  teamId  teamName     submissionDate       score          
--------  -----------  -------------------  -------------  
11836327  3 Radon      2024-03-30 22:13:31  23.5660461342  
11836788  5 Radon☢️    2024-03-31 15:09:08  26.2279319121  
11832968  4 Radon      2024-03-31 20:27:09  26.7874450843  
11830971  7 Radon☢️    2024-04-01 00:00:10  28.8268501431  
11866045  6 Radon      2024-03-31 21:50:04  43.3740939866  
11842717  8 Radon      2024-04-01 01:02:57  44.3920426344  
11849774  1 Radon      2024-04-01 00:32:09  53.4017785555  
11803531  2 Radon      2024-03-31 23:55:31  62.2007604199  
11530127  ☢️Baseline🐍  2023-09-20 18:38:17  81.1731434934  


In [ ]:
!pip install -U tfds-nightly tensorflow_addons tensorflow keras==2.12.0 uszipcode >> log # downgrade keras to bypass tf_utils import error

In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, tensorflow_addons as tfa, tensorflow as tf, tensorflow.keras as keras, os
from keras.layers import Flatten, Dense
os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'; # allows seeding RNG on GPU
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 484 ms, sys: 3.18 ms, total: 487 ms
Wall time: 546 ms


In [ ]:
df_raw = pd.read_csv('XY_radon.csv'); df_raw

,Uppm,adjwt,basement,cntyfips,county,dupflag,floor,lat,lon,pcterr,...,stfips,stopdt,stoptm,stratum,typebldg,wave,windoor,zip,zipflag,Y
0,1.80,54.97,Y,59,MORTON,0,1,46.66,-101.39,7.76,...,38,32206,1230,2,2,1,NaN,58554,0,NaN
1,1.65,499.34,N,85,KOSCIUSKO,0,1,40.85,-86.22,55.02,...,18,11497,1430,3,1,32,NaN,46580,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12571,0.44,394.07,Y,3,ANOKA,0,0,44.91,-92.86,9.40,...,27,32110,1500,2,1,4,NaN,55303,0,8.6
12572,2.71,157.82,Y,15,MOHAVE,0,0,36.01,-113.21,14.46,...,4,11496,1330,1,1,38,NaN,86403,0,1.9


In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=green size=40>

<strong><font color=green size=5>⏳Timed Green Playground (TGP): Your ideas, code, documentation, and timer START HERE!</font></strong>

<font color=green>Students: Keep all your definitions, code, documentation in <b>TGP</b>. Modifying any code outside of TGP incurs penalties.

<font color=green><h3><b>$\alpha$. Split observations into train and test sets</b><h3>

In [ ]:
df = df_raw.select_dtypes(include=np.number).drop('windoor', axis=1)  # drop non-numeric columns
vX = df.query('Y!=Y').drop('Y', axis=1)    # slice a test sample
tXY = df.query('Y==Y')                     # slice training sample
tX, tY = tXY.drop('Y', axis=1), tXY.Y.astype(int)      # split into training I/O
print(tY.tolist()[:50])                    # train outputs

[0, 6, 17, 0, 7, 4, 10, 3, 2, 1, 1, 2, 0, 1, 1, 3, 1, 5, 7, 3, 6, 15, 21, 1, 16, 1, 22, 6, 2, 1, 0, 5, 0, 0, 2, 1, 0, 1, 0, 0, 0, 2, 4, 4, 7, 2, 3, 2, 0, 1]


In [ ]:
tXY.columns

Index(['Uppm', 'adjwt', 'cntyfips', 'dupflag', 'floor', 'lat', 'lon', 'pcterr',
       'region', 'rep', 'room', 'startdt', 'starttm', 'stfips', 'stopdt',
       'stoptm', 'stratum', 'typebldg', 'wave', 'zip', 'zipflag', 'Y'],
      dtype='object')

In [ ]:

# import matplotlib.pyplot as plt
# # plot histogram for each column
# for col in tXY.columns:
#     _ = plt.hist(tXY[col])
#     _ = plt.title(col)
#     _ = plt.show()

In [ ]:
# plot lon, lat with gradient by Y, half transparent
# plt.figure(figsize=(8,6))
# plt.scatter(tXY.lon, tXY.lat, c=tXY.Y, s=tXY.Y/4, cmap='jet', alpha=0.3)
# plt.colorbar()
# plt.show()

In [ ]:
# plt.figure(figsize=(8,6))
# col1 = "pcterr"
# col2 = "Y"
# plt.scatter(1/(tXY[col1]**2 + 1), tXY[col2], c=tXY.stfips.astype(int), alpha=0.1)
# plt.xlabel(col1)
# plt.ylabel(col2)
# plt.show()

In [ ]:
# plt.figure(figsize=(8,6))
# col1 = "pcterr"
# col2 = "Y"
# plt.scatter(tXY[col1].apply(np.log1p), tXY[col2], c=tXY.stfips.astype(int), alpha=0.1)
# plt.xlabel(col1)
# plt.ylabel(col2)
# plt.show()

In [ ]:
# plt.figure(figsize=(8,6))
# col1 = "adjwt"
# col2 = "Y"
# plt.scatter(tXY['stopdt'] - tXY['startdt'], tXY[col2], c=tXY.stfips.astype(int), alpha=0.1)
# plt.xlabel(col1)
# plt.ylabel(col2)
# plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

VALIDATION = False

early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=10,         # How many epochs to wait after val_loss has stopped improving
    verbose=1,
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored metric.
)

# Feature Engineering
tX['pcterr_inv'] = 1/(tX['pcterr']**1 + 1)
tX['pcterr_log'] = tX['pcterr'].apply(np.log1p)
tX['is_zero'] = tX['pcterr_inv'] == 1
tX['stfips_cntyfips'] = tX['stfips'] * 1e3 + tX['cntyfips']

vX['pcterr_inv'] = 1/(vX['pcterr']**1 + 1)
vX['pcterr_log'] = vX['pcterr'].apply(np.log1p)
vX['is_zero'] = vX['pcterr_inv'] == 1
vX['stfips_cntyfips'] = vX['stfips'] * 1e3 + vX['cntyfips']

# Define the preprocessor
numerical_features = ['pcterr_inv', 'pcterr_log', 'stfips_cntyfips']
categorical_features = ['is_zero', 'zipflag', 'rep', 'room', 'typebldg', 'dupflag']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Preprocess the features
preprocessor.fit(pd.concat([tX, vX]))
tX_processed = preprocessor.transform(tX)
tX_processed = tX_processed.toarray() if hasattr(tX_processed, "toarray") else tX_processed
vX_processed = preprocessor.transform(vX)
vX_processed = vX_processed.toarray() if hasattr(vX_processed, "toarray") else vX_processed

if VALIDATION:
    train_x, test_x, train_y, test_y = \
        train_test_split(tX_processed, tY, test_size=0.1, random_state=42, stratify=tX.stfips)

# always seed your experiments
tf.random.set_seed(0)
Init = keras.initializers.glorot_normal(seed=0)

# Hyper Params
learning_rate = 5e-4
epochs = 50
batch_size = 16

# Define Model here
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

m = keras.models.Sequential([
  Flatten(input_shape=[tX_processed.shape[1]]),
  Dense(128, activation="selu", kernel_initializer=Init),
  #BatchNormalization(),
  Dense(128, activation="selu", kernel_initializer=Init),
  #BatchNormalization(),
  Dense(1, kernel_initializer=Init)])
m.summary()
m.compile(loss="mse", optimizer=optimizer, metrics=['mse'])

# Train the model
if VALIDATION:
    hist = m.fit(train_x, train_y,
                 batch_size=batch_size, epochs=epochs,
                 validation_data=(test_x, test_y),
    callbacks=[early_stopping] )
else:
    hist = m.fit(tX_processed, tY,
                 batch_size=batch_size,
                 epochs=epochs,
    callbacks=[early_stopping] )

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['pcterr_inv', 'pcterr_log',
                                  'stfips_cntyfips']),
                                ('cat', OneHotEncoder(),
                                 ['is_zero', 'zipflag', 'rep', 'room',
                                  'typebldg', 'dupflag'])])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 29)                0         
                                                                 
 dense_6 (Dense)             (None, 128)               3840      
                                                                 
 dense_7 (Dense)             (None, 128)               16512     
                                                                 
 dense_8 (Dense)             (None, 1)                 129       
                                                                 
Total params: 20,481
Trainable params: 20,481
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
389/393 [============================>.] - ETA: 0s - loss: 72.2043 - mse: 72.2043

393/393 [==============================] - 2s 2ms/step - loss: 76.9670 - mse: 76.9670
Epoch 2/50
373/393 [===========================>..] - ETA: 0s - loss: 66.0270 - mse: 66.0270

393/393 [==============================] - 1s 3ms/step - loss: 63.1379 - mse: 63.1379
Epoch 3/50
391/393 [============================>.] - ETA: 0s - loss: 51.7486 - mse: 51.7486

393/393 [==============================] - 1s 3ms/step - loss: 51.5272 - mse: 51.5272
Epoch 4/50
382/393 [============================>.] - ETA: 0s - loss: 44.0195 - mse: 44.0195

...

393/393 [==============================] - 1s 1ms/step - loss: 21.7435 - mse: 21.7435
Epoch 47/50
362/393 [==========================>...] - ETA: 0s - loss: 23.6051 - mse: 23.6051

393/393 [==============================] - 1s 2ms/step - loss: 22.3717 - mse: 22.3717
Epoch 48/50
366/393 [==========================>...] - ETA: 0s - loss: 10.0870 - mse: 10.0870

393/393 [==============================] - 1s 2ms/step - loss: 22.0945 - mse: 22.0945
Epoch 49/50
381/393 [============================>.] - ETA: 0s - loss: 22.5142 - mse: 22.5142

393/393 [==============================] - 1s 2ms/step - loss: 21.9734 - mse: 21.9734
Epoch 50/50
382/393 [============================>.] - ETA: 0s - loss: 21.9256 - mse: 21.9256

393/393 [==============================] - 1s 2ms/step - loss: 21.7099 - mse: 21.7099


<font color=green><h3><b>$\beta$. Build and train a model</b><h3>

The model generates a baseline submission CSV file, see Colab folder (🗀 on the left), which you candownload and submit to Kaggle.

<font color=green><h3><b>$\gamma$. Make predictions</b><h3>

In [ ]:
pY = pd.DataFrame(m.predict(vX_processed), index=np.arange(len(vX_processed))+1, columns=['y'])
ToCSV(pY.round(0).astype(int), 'friday')

197/197 [==============================] - 0s 1ms/step


In [ ]:
# !kaggle competitions submit -c 11mar24jh-radon -f friday.csv -m "Message"

<font color=green><h3><b>$\delta$. Documentation</b></h3></font>

<font color=green><h4><b>Task 1. Explain Decisions in Preprocessing Pipeline</b></h4></font>

<font color=green>
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.</font>

<font color=green>

1. Why did you choose these elements? (Something in EDA, prior experience,...? Note: EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

</font>

1) We initially looked at histograms and various scatter plots of our radon data to understand the data and figure out preprocessing strategy. From these, we decided that there are numerical and categorical features that would benefit from separate preprocessing pipelines. Further reading about the dataset and the topic in general from the two sources provided (radon TF dataset and mixed effect regression modeling), we got the general idea that state level and county level data is important. So, we decided to observe data at state level. The conclusions we drew were 1) ground and first floor were important 2) At county level, readings were not uniform 3) at state level, there was a gaussian distribution of radon.  Our ML pipeline includes feature engineering, data transformation, and splitting for validation purposes, followed by defining and training a neural network model with early stopping to prevent overfitting.
The key part of feature engineering includes adding new features based on existing data, such as pcterr_inv, pcterr_log, and a combination of stfips and cntyfips to form stfips_cntyfips. These transformations aim to improve model performance by providing more informative inputs. For instance, taking the inverse and log of pcterr helped in normalizing the distribution of errors and highlighting their magnitudes differently, making patterns more discernible for the DNN model learning. Combining stfips and cntyfips into a single feature helped the model better capture geographic influences by creating a unique identifier for each location.
We futher employed a ColumnTransformer to apply different preprocessing strategies to numerical and categorical features. Numerical features are standardized, which is crucial for neural networks to ensure that all inputs are on a similar scale, aiding in the convergence of the model. Categorical features undergo one-hot encoding, turning them into a format that can be effectively used by the model by representing categorical data as binary vectors. This is particularly important for non-ordinal categorical features where no intrinsic order exists. The decision to split the dataset into training and validation subsets (if VALIDATION is True) with stratification ensures that both subsets are representative of the overall dataset, which is critical for evaluating model performance accurately.

2) We evaluated the effectiveness of various configuration looking at MSE and comparing the results.

3) We tried Principal Component Analysis (PCA) and Linear Discriminant Analysis (LDA) in our data preprocessing pipeline to see if it can improve our machine learning models, especially with one-hot encoding, we were concerned over high dimensionality of data. We tried to reduce the number of input features while retaining the most informative aspects of the data. However, the applying these technique didn't result in improvements to our models. We also tried word embedding instead of onehot encoding to reserve word similarities and experiment whether it adds any important information for our training process, but it didn't yield significant improvement.

<font color=green><h4><b>Task 2. Explain Decisions in Modeling Pipeline</b></h4></font>

<font color=green>
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

</font>

1) Based on our preprocessing decision, we used two separate preprocessing pipelines for categorical and numerical data. The combined data is then fed into DNN. The neural network architecture we used consists of a sequential model with densely connected layers and SELU activations, designed to process the transformed feature vectors. The use of SELU (Scaled Exponential Linear Unit) activations allowed us to leverage its self-normalizing properties, which led to slightly better performance in deep learning models. We also experimented with various regularization and normalization methods such as drops and batch normalization.

2) We evaluated the performance of these various model configurations using mean squared error (MSE) of trained model against test data.

3) In our machine learning project, we tried many different preprocessing steps, feature sets, and model configurations. We added more complex layers to the model, experimenting with different activation functions, applying dropout or batch normalization for regularization, and explored different optimization algorithms such as AdaGrad and various learning rates were explored to improve model performance. The choice of techniques that worked was guided by the feedback loop provided by evaluating model performance after each experiment.

<font color=green><h3><b>$\epsilon$. References</b></h3></font>

references:

1. Radon Dataset Description (https://www.tensorflow.org/datasets/catalog/radon)

2. Multilevel Modeling Primer (https://www.tensorflow.org/probability/examples/Multilevel_Modeling_Primer)

3. Tensorflow Probability Examples (https://github.com/tensorflow/probability/blob/main/tensorflow_probability/examples/jupyter_notebooks/HLM_TFP_R_Stan.ipynb)

4. Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift (https://arxiv.org/abs/1502.03167)

5. Dropout: A Simple Way to Prevent Neural Networks from Overfitting (https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf)

6. Feature Scaling in Machine Learning (https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/)

7. Early Stopping: But When? (https://link.springer.com/chapter/10.1007/978-3-642-35289-8_5)

<font size=5>⌛</font> <strong><font color=green size=5>Do not exceed competition's runtime limit! Do not write code outside TGP</font></strong>
<hr color=green size=40>

In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 84 sec > 60 sec limit!!!


# **Starter Ideas**

1. Try different DNN architectures and tuning of hyperparameters
1. Try converting locations to distances to the key Radon sources (which you might need to discover).
1. Try clustering categorical variables by their relation to Radon levels
1. Try replacing categorical values with their level frequencies or other encodings
1. Try scaling features linearly or nonlinearly
1. Try embedding **textual** values (eg. US States names) with pre-trained SBERT-like models. This injects some additional information from Wikipedia (or whichever corpora were used for model training).
1. Do EDA and understand the variables and their relation to the output. [Example 1](https://docs.pymc.io/en/v3.11.4/pymc-examples/examples/case_studies/multilevel_modeling.html), [Example 2](https://www.tensorflow.org/probability/examples/Multilevel_Modeling_Primer)

<hr>
<font color=black>
    <details><summary><font color=carnelian>▶ </font>Clustering categorical variables <b></b>.</summary>

  1. When we represent categorical variables as dummies, we may be losing important multivariate information. For example, say we use weekdays to predict the number of hours a person works. We could convert weekdays to 6 features (one is dropped due to collinearity). This requires 6 coefficients (degrees of freedom or sources of uncertainty). Essentially, we have an overparameterized model, whereas all we really need is two clusters of categorical values - weekends (Sat/Sun) and non-weekends (M/T/W/Th/F). In general, the model overparameterized model will do worse due to higher variance of the model output (resulting from the overfit and higher flexibility).

  1. Here is another example from the NLP domain, where each word is a feature (or dimension). While morphological variants of a word (eg. run, running, runner, ran, runs, ...) have lower frequency, we cluster them into the same lemma "run", assuming only a small loss of semantic information. We hope that the gain in building a better distribution estimate for "run" is greater than the loss of semantic and lexical information.
        </details>
    <details><summary><font color=carnelian>▶ </font>Distance to Radon source<b></b>.</summary>

If you can determine where Radon is most active (i.e. the source), then you might be able to compute the distance to the source. Ordinarily, we expect lower radiation for greater distance from the source (assuming uniform distribution of underground rivers, geology, rains/winds and other weather conditions affecting distribution of radon, etc.). You could also use categorical features in (e.g. US State, region, etc.), but these might perform better when clustered (again). Distance to the source is a real-valued feature, which does not require clustering.
        </details>
</font>